In [ ]:
import tkinter as tk
from tkinter import filedialog, simpledialog
from PIL import Image, ImageTk, ImageOps, ImageFilter

# Global variables to hold image data and cropping coordinates
img = None
original_img = None  # To store the original image for resetting
img_display = None
crop_coords = None
rect_id = None

def open_image():
    global img, original_img, img_display
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        img = Image.open(file_path)
        original_img = img.copy()  # Store the original image
        img_display = img.copy()
        show_images()

def show_images():
    """Displays both the original and the modified images."""
    if original_img and img_display:
        # Show original image on left canvas
        canvas_original.config(width=original_img.width, height=original_img.height)
        img_original_tk = ImageTk.PhotoImage(original_img)
        canvas_original.create_image(0, 0, anchor="nw", image=img_original_tk)
        canvas_original.img_original_tk = img_original_tk

        # Show modified image on right canvas
        canvas_modified.config(width=img_display.width, height=img_display.height)
        img_modified_tk = ImageTk.PhotoImage(img_display)
        canvas_modified.create_image(0, 0, anchor="nw", image=img_modified_tk)
        canvas_modified.img_modified_tk = img_modified_tk

def reset_image():
    global img_display
    img_display = original_img.copy()  # Reset to the original image
    show_images()

def start_crop(event):
    global crop_coords, rect_id
    crop_coords = [event.x, event.y, event.x, event.y]
    rect_id = canvas_modified.create_rectangle(*crop_coords, outline='red')

def update_crop(event):
    global crop_coords
    if crop_coords:
        crop_coords[2], crop_coords[3] = event.x, event.y
        canvas_modified.coords(rect_id, *crop_coords)

def end_crop(event):
    global img_display, crop_coords
    if crop_coords:
        x_start, y_start, x_end, y_end = crop_coords
        # Ensure coordinates are in the correct order
        x_start, x_end = sorted([x_start, x_end])
        y_start, y_end = sorted([y_start, y_end])
        img_display = img_display.crop((x_start, y_start, x_end, y_end))
        show_images()
        crop_coords = None

def convert_color(option):
    global img_display
    if option == 'Grayscale':
        img_display = ImageOps.grayscale(img_display)
    elif option == 'Black & White':
        img_display = img_display.convert('L').point(lambda x: 0 if x < 128 else 255, '1')
    else:
        img_display = original_img.copy()
    show_images()

def rotate_image():
    global img_display
    angle = simpledialog.askfloat("Rotate", "Enter angle:", minvalue=0, maxvalue=360)
    if angle is not None:
        img_display = img_display.rotate(angle)
        show_images()

def flip_image(flip_type):
    global img_display
    if flip_type == 'Horizontal':
        img_display = img_display.transpose(Image.FLIP_LEFT_RIGHT)
    elif flip_type == 'Vertical':
        img_display = img_display.transpose(Image.FLIP_TOP_BOTTOM)
    show_images()

def apply_filter(filter_type):
    """Applies the selected filter to the displayed image."""
    global img_display
    if filter_type == 'Sharpen':
        img_display = img_display.filter(ImageFilter.SHARPEN)
    elif filter_type == 'Smooth':
        img_display = img_display.filter(ImageFilter.SMOOTH)
    elif filter_type == 'Edge Detection':
        img_display = img_display.filter(ImageFilter.FIND_EDGES)
    elif filter_type == 'Emboss':
        img_display = img_display.filter(ImageFilter.EMBOSS)
    else:
        img_display = original_img.copy()
    show_images()

def save_image():
    file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg"), ("All files", "*.*")])
    if file_path:
        img_display.save(file_path)

# Create main window
root = tk.Tk()
root.title("Image Processing Tool")

# Create frames for side-by-side display
frame_original = tk.Frame(root, padx=5, pady=5)
frame_modified = tk.Frame(root, padx=5, pady=5)
frame_original.pack(side=tk.LEFT)
frame_modified.pack(side=tk.LEFT)

# Create canvases to display images
canvas_original = tk.Canvas(frame_original, cursor="cross", bg="white")
canvas_modified = tk.Canvas(frame_modified, cursor="cross", bg="white")
canvas_original.pack()
canvas_modified.pack()

# Bind mouse events to the right canvas for interactive cropping
canvas_modified.bind("<ButtonPress-1>", start_crop)
canvas_modified.bind("<B1-Motion>", update_crop)
canvas_modified.bind("<ButtonRelease-1>", end_crop)

# Create buttons and options
btn_open = tk.Button(root, text="Open Image", command=open_image)
btn_open.pack(side=tk.TOP)

btn_rotate = tk.Button(root, text="Rotate", command=rotate_image)
btn_rotate.pack(side=tk.TOP)

btn_flip_horizontal = tk.Button(root, text="Flip Horizontal", command=lambda: flip_image('Horizontal'))
btn_flip_horizontal.pack(side=tk.TOP)

btn_flip_vertical = tk.Button(root, text="Flip Vertical", command=lambda: flip_image('Vertical'))
btn_flip_vertical.pack(side=tk.TOP)

btn_reset = tk.Button(root, text="Reset to Original", command=reset_image)
btn_reset.pack(side=tk.TOP)

btn_save = tk.Button(root, text="Save Image", command=save_image)
btn_save.pack(side=tk.TOP)

# Option menu for color conversion
color_option = tk.StringVar(value='Color')
color_menu = tk.OptionMenu(root, color_option, 'Color', 'Grayscale', 'Black & White', command=convert_color)
color_menu.pack(side=tk.TOP)

# Filter buttons
btn_sharpen = tk.Button(root, text="Sharpen", command=lambda: apply_filter('Sharpen'))
btn_sharpen.pack(side=tk.TOP)

btn_smooth = tk.Button(root, text="Smooth", command=lambda: apply_filter('Smooth'))
btn_smooth.pack(side=tk.TOP)

btn_edge_detection = tk.Button(root, text="Edge Detection", command=lambda: apply_filter('Edge Detection'))
btn_edge_detection.pack(side=tk.TOP)

btn_emboss = tk.Button(root, text="Emboss", command=lambda: apply_filter('Emboss'))
btn_emboss.pack(side=tk.TOP)

root.mainloop()